In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from datasets import load_dataset
import os
import torch.optim as optim
from torchvision import datasets, transforms, models

In [ ]:
# Access token for the Hugging Face API to download the dataset ACL-fig
access_token = "YOUR_ACCESS"
dataset_name = "citeseerx/ACL-fig"

## Loading the dataset

We provide the chemical formulas dataset as part of this repository. They are `class1` in the `images` folder. The following code loads the `ACL-fig` dataset and copies the content into the local folder structure (as `class2`).

In [ ]:
# Dataset ACL-fig-Dataset as base, enriched with our own samples
dataset = load_dataset(dataset_name, token=access_token)

print(f"Dataset loaded, number of samples: {len(dataset)}")

In [ ]:
# Split dataset into train, test, and validation sets
train_dataset = dataset['train'] 
test_dataset = dataset['test']
valid_dataset = dataset['validation']

In [ ]:
import os

def save_images_to_directory(dataset, save_dir):

    """
    Save images from a Hugging Face dataset to a local directory.
    
    Args:
        dataset: Hugging Face dataset containing images
        save_dir (str): Directory path where images will be saved
    """

    # Create directory if it doesn't exist
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    # Save all images from the HF dataset to the specified directory
    for i, sample in enumerate(dataset):
        # Check if the image is correctly found under the key "image"
        if "image" in sample:
            image = sample["image"]
            # Set the path for the image
            image_save_path = os.path.join(save_dir, f"image_{i}.jpg")
            # Save image as JPEG
            image.save(image_save_path)
            print(f"Image {i} saved: {image_save_path}")
        else:
            print(f"No image found in dataset {i}")

# Define directories for saving train and test images
save_dir_train = "images/train/class2"
save_dir_test = "images/test/class2"

# Save images to respective directories
save_images_to_directory(train_dataset, save_dir_train)
save_images_to_directory(test_dataset, save_dir_test)

print("All images have been successfully downloaded and saved.")

In [ ]:
# Set up device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define image processing parameters
image_size = 224
batch_size = 32 

In [ ]:
# Define image transformations pipeline
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    # Normalize using ImageNet statistics for transfer learning
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Define paths to the train and test data directories
# These directories should contain subdirectories for each class
train_data_dir = './images/train'
test_data_dir = './images/test'

In [ ]:
# Create Dataset objects using ImageFolder
# ImageFolder assumes a directory structure where:
# - Each class has its own subdirectory
# - Images for each class are stored in their respective subdirectory
# The 'transform' parameter applies the previously defined preprocessing pipeline
train_dataset = datasets.ImageFolder(
    root="images/train",
    transform=transform
)
test_dataset = datasets.ImageFolder(
    root="images/test",
    transform=transform

In [ ]:
# Create DataLoader instances for efficient batch processing
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=32,     # Number of samples per batch
    shuffle=True       # Shuffle data during training to prevent learning order dependencies
)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=32,     # Keep batch size consistent with training
    shuffle=False      # No need to shuffle test data
)

## Modeltraining for Binary Classification
We implement a comprehensive training pipeline for binary image classification using multiple state-of-the-art CNN architectures. The code includes a generic training function `train_model` that handles the training process for all models, followed by individual initialization and training of four different architectures: ResNet-18, AlexNet, EfficientNet-B0, and RegNet-Y-400MF.
Each model is pre-trained on ImageNet and modified for our binary classification task by adjusting the final classification layer. The training process includes:

- 10 epochs of training for each model
- Adam optimizer with a learning rate of 0.001
- CrossEntropyLoss as the loss function
- Automatic GPU utilization when available
- Real-time loss tracking and final accuracy evaluation

The models are trained sequentially, allowing for direct comparison of their performance on the chemical formulas (`class1`) and ACL-fig (`class2`) dataset. Each model's training progress and final test accuracy are logged for performance analysis.

In [ ]:
def train_model(model_name, model, num_epochs=10, learning_rate=0.001):

    """
    Generic training function for image classification models

    Args:
        model_name (str): Name identifier for the model being trained
        model (torch.nn.Module): Neural network model to be trained
        num_epochs (int): Number of complete passes through the training dataset
        learning_rate (float): Step size for optimizer updates

    Returns:
        model: Trained PyTorch model
    """
    
    # Move model to device (GPU if available, else CPU)
    model = model.to(device)

    # Initialize loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Main training loop - iterate through epoch
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
        # Print training progress after each epoch
        print(f"{model_name} - Epoch [{epoch+1}/{num_epochs}], "
              f"Loss: {running_loss/len(train_loader):.4f}")
    
    # Evaluation phase - test model performance
    model.eval()
    correct = 0
    total = 0
    
    # Disable gradient computation for evaluation
    with torch.no_grad():
        # Iterate through test dataset
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)       # Get predicted class labels
            total += labels.size(0)                         # Count total samples
            correct += (predicted == labels).sum().item()   # Count correct predictions
    
    # Print final accuracy
    print(f"{model_name} Test Accuracy: {100 * correct / total:.2f}%")
    return model

#### ResNet-18

In [ ]:
# Initialize ResNet-18
resnet18_model = models.resnet18(pretrained=True)
num_ftrs = resnet18_model.fc.in_features
resnet18_model.fc = nn.Linear(num_ftrs, 2)

In [ ]:
# Train ResNet-18
print("\nTraining ResNet-18...")
trained_resnet = train_model(
    model_name="ResNet-18",
    model=resnet18_model,
    num_epochs=10,
    learning_rate=0.001
)

#### AlexNet

In [ ]:
# Initialize AlexNet
alexnet_model = models.alexnet(pretrained=True)
num_ftrs = alexnet_model.classifier[6].in_features
alexnet_model.classifier[6] = nn.Linear(num_ftrs, 2)

In [ ]:
# Train AlexNet
print("\nTraining AlexNet...")
trained_alexnet = train_model(
    model_name="AlexNet",
    model=alexnet_model,
    num_epochs=10,
    learning_rate=0.001
)

#### EfficientNet

In [ ]:
# Initialize EfficientNet
efficientnet_model = models.efficientnet_b0(pretrained=True)
num_ftrs = efficientnet_model.classifier[1].in_features
efficientnet_model.classifier[1] = nn.Linear(num_ftrs, 2)

In [ ]:
# Train EfficientNet
print("\nTraining EfficientNet-B0...")
trained_efficientnet = train_model(
    model_name="EfficientNet-B0",
    model=efficientnet_model,
    num_epochs=10,
    learning_rate=0.001
)

#### RegNet-Y-400MF

In [ ]:
# Initialize RegNet
regnet_model = models.regnet_y_400mf(pretrained=True)
num_ftrs = regnet_model.fc.in_features
regnet_model.fc = nn.Linear(num_ftrs, 2)

In [ ]:
# Train RegNet
print("\nTraining RegNet-Y-400MF...")
trained_regnet = train_model(
    model_name="RegNet-Y-400MF",
    model=regnet_model,
    num_epochs=10,
    learning_rate=0.001
)